# SQL: JoinQueries

## Setup

We are now installing the necessary packages to interact with the MySQL database and issue SQL queries using the notebook.

In [1]:
!sudo apt-get install python3-mysqldb
!sudo pip3 install -U sqlalchemy sql_magic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-mysqldb is already the newest version (1.3.10-1build1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Requirement already up-to-date: sqlalchemy in /usr/local/lib/python3.6/dist-packages (1.3.16)
Requirement already up-to-date: sql_magic in /usr/local/lib/python3.6/dist-packages (0.0.4)


In [7]:
%reload_ext sql_magic

In [0]:
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/?charset=utf8'.format(
    host='db.ipeirotis.org',
    user='student',
    password='dwdstudent2015',
    encoding='utf-8')
engine = create_engine(conn_string)

In [0]:
%config SQL.conn_name = 'engine'

## Poor man's joins: Find the genres that Steven Spielberg typically directs

Use the table directors_genres  and find all genres of films and the corresponding probabilities for the director ID that corresponds to Steven Spielberg. Sort the results by probability. 


#### Switch to IMDb

In [59]:
%%read_sql
USE imdb

Query started at 03:08:30 AM UTC; Query executed in 0.03 m

#### Find the entry for Steven Spielberg to find his id

In [60]:
%%read_sql
SELECT *
FROM directors 
WHERE first_name = 'Steven' AND last_name = 'Spielberg'

Query started at 03:08:31 AM UTC; Query executed in 0.01 m

,id,first_name,last_name
0,75380,Steven,Spielberg


#### Query the `directors_genres` table for Spielberg movies (pick director id value from previous query)

In [61]:
%%read_sql
SELECT *
FROM directors_genres
WHERE director_id = 75380
ORDER BY prob DESC

Query started at 03:08:32 AM UTC; Query executed in 0.01 m

,director_id,genre,prob
0,75380,Drama,0.435897
1,75380,Thriller,0.282051
2,75380,Adventure,0.282051
3,75380,Sci-Fi,0.256410
4,75380,Action,0.230769
5,75380,Horror,0.179487
6,75380,Short,0.128205
7,75380,Fantasy,0.102564
8,75380,War,0.102564
9,75380,Crime,0.102564


## Simple Join Queries

#### Switch to IMDb

In [62]:
%%read_sql
USE imdb

Query started at 03:08:32 AM UTC; Query executed in 0.01 m

#### List all the movies and their genres

In [63]:
%%read_sql
SELECT *
FROM movies M 
  INNER JOIN movies_genres G ON M.id = G.movie_id


Query started at 03:08:33 AM UTC; Query executed in 0.05 m

,id,name,year,rank,movie_id,genre
0,1,"#7 Train: An Immigrant Journey, The",2000,NaN,1,Short
1,1,"#7 Train: An Immigrant Journey, The",2000,NaN,1,Documentary
2,2,$,1971,6.4,2,Crime
3,2,$,1971,6.4,2,Comedy
4,5,"$1,000 Reward",1923,NaN,5,Western
...,...,...,...,...,...,...
395445,378612,nz de mihlarim,1965,NaN,378612,Adventure
395446,378612,nz de mihlarim,1965,NaN,378612,Drama
395447,378613,egar a gerist,1998,NaN,378613,Comedy
395448,378613,egar a gerist,1998,NaN,378613,Drama


#### List the movie genres for the movies directed by Steven Spielberg and sort them in decreasing order of their probability (use the director_genres table)



In [64]:
%%read_sql
SELECT *
FROM directors D
  INNER JOIN directors_genres G ON G.director_id = D.id
WHERE D.first_name = 'Steven' AND D.last_name = 'Spielberg'
ORDER BY G.prob DESC

Query started at 03:08:36 AM UTC; Query executed in 0.01 m

,id,first_name,last_name,director_id,genre,prob
0,75380,Steven,Spielberg,75380,Drama,0.435897
1,75380,Steven,Spielberg,75380,Thriller,0.282051
2,75380,Steven,Spielberg,75380,Adventure,0.282051
3,75380,Steven,Spielberg,75380,Sci-Fi,0.256410
4,75380,Steven,Spielberg,75380,Action,0.230769
5,75380,Steven,Spielberg,75380,Horror,0.179487
6,75380,Steven,Spielberg,75380,Short,0.128205
7,75380,Steven,Spielberg,75380,War,0.102564
8,75380,Steven,Spielberg,75380,Fantasy,0.102564
9,75380,Steven,Spielberg,75380,Crime,0.102564


In [65]:
%%read_sql
SELECT G.genre, G.prob
FROM directors D
  INNER JOIN directors_genres G ON G.director_id = D.id
WHERE D.first_name = 'Steven' AND D.last_name = 'Spielberg'
ORDER BY G.prob DESC

Query started at 03:08:36 AM UTC; Query executed in 0.01 m

,genre,prob
0,Drama,0.435897
1,Thriller,0.282051
2,Adventure,0.282051
3,Sci-Fi,0.256410
4,Action,0.230769
5,Horror,0.179487
6,Short,0.128205
7,Fantasy,0.102564
8,War,0.102564
9,Crime,0.102564


#### List all the movies and their directors

In [66]:
%%read_sql
SELECT *
FROM directors D
  INNER JOIN movies_directors MD ON MD.director_id = D.id
  INNER JOIN movies M ON M.id = MD.movie_id

Query started at 03:08:37 AM UTC; Query executed in 0.20 m

,id,first_name,last_name,director_id,movie_id,id,name,year,rank
0,1,Todd,1,1,378879,378879,"""5 Deadly Videos""",2004,NaN
1,2,Les,12 Poissons,2,281325,281325,Rondo pour trompette,2002,NaN
2,3,Lejaren,a'Hiller,3,30621,30621,"Bashful Suitor, The",1921,NaN
3,3,Lejaren,a'Hiller,3,304743,304743,"Sleep of Cyma Roget, The",1920,NaN
4,4,Nian,A,4,60570,60570,Chengshi aiqing,1997,NaN
...,...,...,...,...,...,...,...,...,...
380655,88797,Yusuf,Ünal,88797,172648,172648,Kalbim agliyor,1988,NaN
380656,88798,Ahmet,Ündag,88798,350996,350996,Ve gnese kan siradi,1972,NaN
380657,88799,Idil,Üner,88799,189713,189713,"Liebenden vom Hotel von Osman, Die",2001,8.5
380658,88800,Yüksel,Ünsal,88800,105513,105513,Evvel zaman iinde,1951,NaN


In [67]:
%%read_sql
SELECT MD.director_id, D.first_name AS director_first_name, D.last_name AS director_last_name,
       MD.movie_id, M.name AS movie_title, M.year AS release_year, M.rank AS movie_rating 
FROM directors D
  INNER JOIN movies_directors MD ON MD.director_id = D.id
  INNER JOIN movies M ON M.id = MD.movie_id

Query started at 03:08:49 AM UTC; Query executed in 0.18 m

,director_id,director_first_name,director_last_name,movie_id,movie_title,release_year,movie_rating
0,1,Todd,1,378879,"""5 Deadly Videos""",2004,NaN
1,2,Les,12 Poissons,281325,Rondo pour trompette,2002,NaN
2,3,Lejaren,a'Hiller,30621,"Bashful Suitor, The",1921,NaN
3,3,Lejaren,a'Hiller,304743,"Sleep of Cyma Roget, The",1920,NaN
4,4,Nian,A,60570,Chengshi aiqing,1997,NaN
...,...,...,...,...,...,...,...
380655,88797,Yusuf,Ünal,172648,Kalbim agliyor,1988,NaN
380656,88798,Ahmet,Ündag,350996,Ve gnese kan siradi,1972,NaN
380657,88799,Idil,Üner,189713,"Liebenden vom Hotel von Osman, Die",2001,8.5
380658,88800,Yüksel,Ünsal,105513,Evvel zaman iinde,1951,NaN


#### List all the movies directed by Steven Spielberg

In [68]:
%%read_sql
SELECT MD.director_id, D.first_name AS director_first_name, D.last_name AS director_last_name,
       MD.movie_id, M.name AS movie_title, M.year AS release_year, M.rank AS movie_rating 
FROM directors D
  INNER JOIN movies_directors MD ON MD.director_id = D.id
  INNER JOIN movies M ON M.id = MD.movie_id
WHERE D.first_name = 'Steven' AND D.last_name = 'Spielberg'
ORDER BY M.rank DESC

Query started at 03:08:59 AM UTC; Query executed in 0.01 m

,director_id,director_first_name,director_last_name,movie_id,movie_title,release_year,movie_rating
0,75380,Steven,Spielberg,103334,Escape to Nowhere,1961,9.1
1,75380,Steven,Spielberg,113931,Firelight,1964,9.0
2,75380,Steven,Spielberg,290070,Schindler's List,1993,8.8
3,75380,Steven,Spielberg,271095,Raiders of the Lost Ark,1981,8.7
4,75380,Steven,Spielberg,305079,Slipstream,1967,8.6
5,75380,Steven,Spielberg,13547,Amblin',1968,8.4
6,75380,Steven,Spielberg,185306,"Last Gun, The",1959,8.4
7,75380,Steven,Spielberg,289109,Saving Private Ryan,1998,8.3
8,75380,Steven,Spielberg,165961,Jaws,1975,8.2
9,75380,Steven,Spielberg,159172,Indiana Jones and the Last Crusade,1989,8.0


## Join Practice Queries: Drama movies from 2000


#### Switch to IMDb

In [69]:
%%read_sql
USE imdb

Query started at 03:09:00 AM UTC; Query executed in 0.01 m

#### List all the movies from year 2000



In [70]:
%%read_sql
SELECT *
FROM movies
WHERE year=2000

Query started at 03:09:00 AM UTC; Query executed in 0.01 m

,id,name,year,rank
0,1,"#7 Train: An Immigrant Journey, The",2000,NaN
1,31,$pent,2000,4.3
2,34,& frres,2000,NaN
3,46,"'70s: The Decade That Changed Television, The",2000,NaN
4,176,'N Sync & Britney Spears: Your #1 Video Reques...,2000,NaN
...,...,...,...,...
11638,412155,"""Quin dijo miedo?""",2000,NaN
11639,412218,"""ndernes magt""",2000,NaN
11640,412221,"""pent hus""",2000,NaN
11641,412232,"""a s'en va et a revient""",2000,NaN


#### List all the movies from year 2000 and their genres

In [71]:
%%read_sql
SELECT *
FROM movies M
  INNER JOIN movies_genres G ON G.movie_id = M.id
WHERE M.year=2000

Query started at 03:09:01 AM UTC; Query executed in 0.01 m

,id,name,year,rank,movie_id,genre
0,1,"#7 Train: An Immigrant Journey, The",2000,NaN,1,Short
1,1,"#7 Train: An Immigrant Journey, The",2000,NaN,1,Documentary
2,31,$pent,2000,4.3,31,Drama
3,34,& frres,2000,NaN,34,Short
4,46,"'70s: The Decade That Changed Television, The",2000,NaN,46,Documentary
...,...,...,...,...,...,...
12138,378537,ber die Montage,2000,NaN,378537,Short
12139,378550,"berfall, Der",2000,7.0,378550,Comedy
12140,378550,"berfall, Der",2000,7.0,378550,Crime
12141,378550,"berfall, Der",2000,7.0,378550,Drama


#### List all the Drama movies from year 2000

In [72]:
%%read_sql
SELECT *
FROM movies M
  INNER JOIN movies_genres G ON G.movie_id = M.id
WHERE M.year=2000 AND G.genre = 'Drama'

Query started at 03:09:02 AM UTC; Query executed in 0.02 m

,id,name,year,rank,movie_id,genre
0,31,$pent,2000,4.3,31,Drama
1,428,...Or Forever Hold Your Peace,2000,6.9,428,Drama
2,598,1. de ei,2000,NaN,598,Drama
3,992,12 Rounds,2000,NaN,992,Drama
4,1023,12x12,2000,NaN,1023,Drama
...,...,...,...,...,...,...
1864,377490,"a, c'est vraiment toi",2000,NaN,377490,Drama
1865,378026,"tre chair, L'",2000,NaN,378026,Drama
1866,378121,skabrn jarinnar,2000,4.2,378121,Drama
1867,378308,tztalmann und seine Welt. Das Jahr bevor er sc...,2000,6.3,378308,Drama


#### List all the Drama movies from year 2000 with ratings

In [73]:
%%read_sql
SELECT *
FROM movies M
  INNER JOIN movies_genres G ON G.movie_id = M.id
WHERE M.year=2000 AND G.genre = 'Drama' AND M.rank IS NOT NULL

Query started at 03:09:03 AM UTC; Query executed in 0.02 m

,id,name,year,rank,movie_id,genre
0,31,$pent,2000,4.3,31,Drama
1,428,...Or Forever Hold Your Peace,2000,6.9,428,Drama
2,1160,15-Sai: Gakko IV,2000,6.5,1160,Drama
3,1351,19,2000,6.8,1351,Drama
4,1784,21,2000,6.5,1784,Drama
...,...,...,...,...,...,...
936,376257,"Zwarte meteoor, De",2000,6.5,376257,Drama
937,376518,Zycie jako smiertelna choroba przenoszona drog...,2000,6.7,376518,Drama
938,378121,skabrn jarinnar,2000,4.2,378121,Drama
939,378308,tztalmann und seine Welt. Das Jahr bevor er sc...,2000,6.3,378308,Drama


#### List the top-50 Drama movies from year 2000, based on the ratings 


In [74]:
%%read_sql
SELECT *
FROM movies M
  INNER JOIN movies_genres G ON G.movie_id = M.id
WHERE M.year=2000 AND G.genre = 'Drama' AND M.rank IS NOT NULL
ORDER BY M.rank DESC
LIMIT 50

Query started at 03:09:04 AM UTC; Query executed in 0.02 m

,id,name,year,rank,movie_id,genre
0,102311,Era outra vez,2000,9.6,102311,Drama
1,104266,Etat,2000,9.3,104266,Drama
2,123371,Game Day,2000,9.2,123371,Drama
3,121732,"Funeral de los dedos, El",2000,9.2,121732,Drama
4,206201,Markova: Comfort Gay,2000,9.1,206201,Drama
5,345028,Un amor de Borges,2000,9.1,345028,Drama
6,249287,Paromitar Ek Din,2000,9.0,249287,Drama
7,46522,Breathe,2000,9.0,46522,Drama
8,289644,Scarlett Mecca and the Pentagram Girl,2000,8.9,289644,Drama
9,70865,Coronacin,2000,8.9,70865,Drama


## Join Practice Queries: James Bond movies

#### Switch to IMDb

In [75]:
%%read_sql
USE imdb

Query started at 03:09:05 AM UTC; Query executed in 0.01 m

#### List all the movies where there is an actor with the role ‘James Bond’


In [76]:
%%read_sql
SELECT *
FROM movies M
  INNER JOIN roles R ON R.movie_id = M.id
WHERE R.role = 'James Bond'

Query started at 03:09:05 AM UTC; Query executed in 0.01 m

,id,name,year,rank,actor_id,movie_id,role
0,21756,Ashakara,1991,NaN,7279,21756,James Bond
1,130945,GoldenEye,1997,NaN,32400,130945,James Bond
2,531,007: Agent Under Fire,2001,NaN,42148,531,James Bond
3,530,007 Racing,2000,NaN,44840,530,James Bond
4,531,007: Agent Under Fire,2001,NaN,44840,531,James Bond
...,...,...,...,...,...,...,...
56,140431,Hawaa wal kerd,1968,NaN,341028,140431,James Bond
57,76256,D4G,2000,NaN,471425,76256,James Bond
58,245547,Outwitted (1917/II),1917,NaN,480224,245547,James Bond
59,123256,Gam gai,2002,7.5,480564,123256,James Bond


#### List the actors who played ‘James Bond’

In [77]:
%%read_sql
SELECT *
FROM actors A 
  INNER JOIN roles R ON R.actor_id = A.id
WHERE R.role = 'James Bond'

Query started at 03:09:06 AM UTC; Query executed in 0.01 m

,id,first_name,last_name,gender,actor_id,movie_id,role
0,7279,Bahoumodom,Ali,M,7279,21756,James Bond
1,32400,Kevin,Bayliss,M,32400,130945,James Bond
2,42148,Andrew,Bicknell,M,42148,531,James Bond
3,44840,Adam,Blackwood,M,44840,530,James Bond
4,44840,Adam,Blackwood,M,44840,531,James Bond
...,...,...,...,...,...,...,...
56,341028,Salah,Nazmi,M,341028,140431,James Bond
57,471425,George (VI),Thomas,M,471425,76256,James Bond
58,480224,Frank,Truesdell,M,480224,245547,James Bond
59,480564,Eric (I),Tsang,M,480564,123256,James Bond


#### List the actors who played ‘James Bond’ and the name of the movie

In [78]:
%%read_sql
SELECT *
FROM actors A 
  INNER JOIN roles R ON R.actor_id = A.id
  INNER JOIN movies M ON M.id = R.movie_id
WHERE R.role = 'James Bond'

Query started at 03:09:06 AM UTC; Query executed in 0.01 m

,id,first_name,last_name,gender,actor_id,movie_id,role,id,name,year,rank
0,7279,Bahoumodom,Ali,M,7279,21756,James Bond,21756,Ashakara,1991,NaN
1,32400,Kevin,Bayliss,M,32400,130945,James Bond,130945,GoldenEye,1997,NaN
2,42148,Andrew,Bicknell,M,42148,531,James Bond,531,007: Agent Under Fire,2001,NaN
3,44840,Adam,Blackwood,M,44840,530,James Bond,530,007 Racing,2000,NaN
4,44840,Adam,Blackwood,M,44840,531,James Bond,531,007: Agent Under Fire,2001,NaN
...,...,...,...,...,...,...,...,...,...,...,...
56,341028,Salah,Nazmi,M,341028,140431,James Bond,140431,Hawaa wal kerd,1968,NaN
57,471425,George (VI),Thomas,M,471425,76256,James Bond,76256,D4G,2000,NaN
58,480224,Frank,Truesdell,M,480224,245547,James Bond,245547,Outwitted (1917/II),1917,NaN
59,480564,Eric (I),Tsang,M,480564,123256,James Bond,123256,Gam gai,2002,7.5


#### List the actors who played ‘James Bond’ and the name of the movie. Rank the result by rating

In [79]:
%%read_sql
SELECT *
FROM actors A 
  INNER JOIN roles R ON R.actor_id = A.id
  INNER JOIN movies M ON M.id = R.movie_id
WHERE R.role = 'James Bond'
ORDER BY M.rank DESC

Query started at 03:09:07 AM UTC; Query executed in 0.01 m

,id,first_name,last_name,gender,actor_id,movie_id,role,id,name,year,rank
0,93861,Sean,Connery,M,93861,130953,James Bond,130953,Goldfinger,1964,7.7
1,480564,Eric (I),Tsang,M,480564,123256,James Bond,123256,Gam gai,2002,7.5
2,93861,Sean,Connery,M,93861,120574,James Bond,120574,From Russia with Love,1963,7.3
3,93861,Sean,Connery,M,93861,92573,James Bond,92573,Dr. No,1962,7.2
4,328803,Roger (I),Moore,M,328803,312170,James Bond,312170,"Spy Who Loved Me, The",1977,6.9
...,...,...,...,...,...,...,...,...,...,...,...
56,59376,Pierce,Brosnan,M,59376,74724,James Bond,74724,Cubby Broccoli: The Man Behind Bond,2000,NaN
57,269903,George,Lazenby,M,269903,160315,James Bond,160315,Inside 'The Man with the Golden Gun',2000,NaN
58,59376,Pierce,Brosnan,M,59376,223076,James Bond,223076,"Music of James Bond, The",2000,NaN
59,269903,George,Lazenby,M,269903,234188,James Bond,234188,Nobody Does It Better: The Music of James Bond,1998,NaN


#### List the actors who played ‘James Bond’ and the name of the movie. Rank the result by year

In [80]:
%%read_sql
SELECT *
FROM actors A 
  INNER JOIN roles R ON R.actor_id = A.id
  INNER JOIN movies M ON M.id = R.movie_id
WHERE R.role = 'James Bond'
ORDER BY M.year

Query started at 03:09:07 AM UTC; Query executed in 0.01 m

,id,first_name,last_name,gender,actor_id,movie_id,role,id,name,year,rank
0,480224,Frank,Truesdell,M,480224,245547,James Bond,245547,Outwitted (1917/II),1917,NaN
1,93861,Sean,Connery,M,93861,92573,James Bond,92573,Dr. No,1962,7.2
2,93861,Sean,Connery,M,93861,120574,James Bond,120574,From Russia with Love,1963,7.3
3,93861,Sean,Connery,M,93861,130953,James Bond,130953,Goldfinger,1964,7.7
4,93861,Sean,Connery,M,93861,332065,James Bond,332065,Thunderball,1965,6.8
...,...,...,...,...,...,...,...,...,...,...,...
56,42148,Andrew,Bicknell,M,42148,531,James Bond,531,007: Agent Under Fire,2001,NaN
57,78159,Maxwell,Caulfield,M,78159,165261,James Bond,165261,James Bond 007: Nightfire,2002,NaN
58,59376,Pierce,Brosnan,M,59376,86263,James Bond,86263,Die Another Day,2002,6.4
59,480564,Eric (I),Tsang,M,480564,123256,James Bond,123256,Gam gai,2002,7.5


## JOIN practice queries: Brad Pitt movies

#### List all the movies where Brad Pitt is playing

In [81]:
%%read_sql
SELECT *
FROM actors A 
  INNER JOIN roles R ON R.actor_id = A.id
  INNER JOIN movies M ON M.id = R.movie_id
WHERE A.first_name = 'Brad' AND A.last_name = 'Pitt'


Query started at 03:09:08 AM UTC; Query executed in 0.01 m

,id,first_name,last_name,gender,actor_id,movie_id,role,id,name,year,rank
0,376249,Brad,Pitt,M,376249,846,Himself - #53: Red Carpet Dis,846,101 Biggest Celebrity Oops,2004,NaN
1,376249,Brad,Pitt,M,376249,1449,Himself,1449,1995 MTV Movie Awards,1995,NaN
2,376249,Brad,Pitt,M,376249,2787,Himself,2787,"54th Annual Primetime Emmy Awards, The",2002,NaN
3,376249,Brad,Pitt,M,376249,2805,Himself,2805,"56th Annual Primetime Emmy Awards, The",2004,NaN
4,376249,Brad,Pitt,M,376249,2818,Himself,2818,"59th Annual Golden Globe Awards, The",2002,NaN
5,376249,Brad,Pitt,M,376249,2924,Himself (Presenter),2924,"67th Annual Academy Awards, The",1995,NaN
6,376249,Brad,Pitt,M,376249,3040,Himself - Presenter: Best Cin,3040,"72nd Annual Academy Awards, The",2000,NaN
7,376249,Brad,Pitt,M,376249,4642,Himself,4642,Abby Singer,2003,4.4
8,376249,Brad,Pitt,M,376249,5855,Joe Maloney,5855,Across the Tracks,1991,5.4
9,376249,Brad,Pitt,M,376249,13751,Himself,13751,America: A Tribute to Heroes,2001,NaN


#### List all the movies where Brad Pitt is playing. Exclude the movies where he plays “himself”



In [82]:
%%read_sql
SELECT *
FROM actors A 
  INNER JOIN roles R ON R.actor_id = A.id
  INNER JOIN movies M ON M.id = R.movie_id
WHERE A.first_name = 'Brad' AND A.last_name = 'Pitt'
  AND R.role NOT LIKE '%%himself%%'

Query started at 03:09:08 AM UTC; Query executed in 0.01 m

,id,first_name,last_name,gender,actor_id,movie_id,role,id,name,year,rank
0,376249,Brad,Pitt,M,376249,5855,Joe Maloney,5855,Across the Tracks,1991,5.4
1,376249,Brad,Pitt,M,376249,69391,"Brad, Bachelor #1",69391,Confessions of a Dangerous Mind,2002,7.1
2,376249,Brad,Pitt,M,376249,69894,Cox,69894,Contact,1992,6.3
3,376249,Brad,Pitt,M,376249,70421,Detective Frank Harris,70421,Cool World,1992,4.1
4,376249,Brad,Pitt,M,376249,78950,Rick,78950,"Dark Side of the Sun, The",1997,4.7
5,376249,Brad,Pitt,M,376249,85050,Rory Devaney/Francis 'Frankie,85050,"Devil's Own, The",1997,5.7
6,376249,Brad,Pitt,M,376249,110170,Elliott Fowler,110170,"Favor, The",1994,5.0
7,376249,Brad,Pitt,M,376249,112290,Tyler Durden,112290,Fight Club,1999,8.5
8,376249,Brad,Pitt,M,376249,134735,Host/Narrator,134735,Growing Up Grizzly,2001,NaN
9,376249,Brad,Pitt,M,376249,138889,Brian,138889,Happy Together (1989/I),1989,5.1


#### List all the movies where Brad Pitt is playing. Exclude the movies where he plays “himself”. Rank the result by movie rating.

In [83]:
%%read_sql
SELECT *
FROM actors A 
  INNER JOIN roles R ON R.actor_id = A.id
  INNER JOIN movies M ON M.id = R.movie_id
WHERE A.first_name = 'Brad' AND A.last_name = 'Pitt'
  AND R.role NOT LIKE '%%himself%%'
ORDER BY M.rank DESC

Query started at 03:09:08 AM UTC; Query executed in 0.01 m

,id,first_name,last_name,gender,actor_id,movie_id,role,id,name,year,rank
0,376249,Brad,Pitt,M,376249,112290,Tyler Durden,112290,Fight Club,1999,8.5
1,376249,Brad,Pitt,M,376249,291698,Detective David Mills,291698,Se7en,1995,8.4
2,376249,Brad,Pitt,M,376249,342384,Jeffrey Goines,342384,Twelve Monkeys,1995,7.9
3,376249,Brad,Pitt,M,376249,306032,Mickey O'Neil,306032,Snatch.,2000,7.9
4,376249,Brad,Pitt,M,376249,340824,Floyd,340824,True Romance,1993,7.7
5,376249,Brad,Pitt,M,376249,238072,Rusty Ryan,238072,Ocean's Eleven,2001,7.5
6,376249,Brad,Pitt,M,376249,329833,J.D.,329833,Thelma & Louise,1991,7.3
7,376249,Brad,Pitt,M,376249,161108,Louis de Pointe du Lac,161108,Interview with the Vampire: The Vampire Chroni...,1994,7.1
8,376249,Brad,Pitt,M,376249,279173,Paul Maclean,279173,"River Runs Through It, A",1992,7.1
9,376249,Brad,Pitt,M,376249,69391,"Brad, Bachelor #1",69391,Confessions of a Dangerous Mind,2002,7.1


#### List all the movies where Brad Pitt is playing. Exclude the movies where he plays “himself”. Rank the result by year.

In [84]:
%%read_sql
SELECT *
FROM actors A 
  INNER JOIN roles R ON R.actor_id = A.id
  INNER JOIN movies M ON M.id = R.movie_id
WHERE A.first_name = 'Brad' AND A.last_name = 'Pitt'
  AND R.role NOT LIKE '%%himself%%'
ORDER BY M.year

Query started at 03:09:09 AM UTC; Query executed in 0.01 m

,id,first_name,last_name,gender,actor_id,movie_id,role,id,name,year,rank
0,376249,Brad,Pitt,M,376249,380391,Chris (1987),380391,"""Another World""",1964,NaN
1,376249,Brad,Pitt,M,376249,233692,Waiter,233692,No Man's Land,1987,5.2
2,376249,Brad,Pitt,M,376249,188135,Partygoer,188135,Less Than Zero,1987,5.5
3,376249,Brad,Pitt,M,376249,315261,Theodore 'Teddy' Johnson,315261,"Stoning in Fulham County, A",1988,NaN
4,376249,Brad,Pitt,M,376249,138889,Brian,138889,Happy Together (1989/I),1989,5.1
5,376249,Brad,Pitt,M,376249,390176,Walker Lovejoy (1990),390176,"""Glory Days""",1990,NaN
6,376249,Brad,Pitt,M,376249,335724,Billy Canton,335724,Too Young to Die?,1990,NaN
7,376249,Brad,Pitt,M,376249,156785,Steve Black,156785,"Image, The",1990,NaN
8,376249,Brad,Pitt,M,376249,5855,Joe Maloney,5855,Across the Tracks,1991,5.4
9,376249,Brad,Pitt,M,376249,168743,Johnny Suede,168743,Johnny Suede,1991,5.8


## JOIN Practice Queries: Facebook

#### Switch to Facebook

In [30]:
%%read_sql
USE facebook;

Query started at 03:52:16 AM UTC; Query executed in 0.01 m

#### List all the Single students



In [86]:
%%read_sql
SELECT *
FROM Profiles P
  INNER JOIN Relationship R ON R.ProfileID = P.ProfileID
WHERE R.Status = 'Single'

Query started at 03:09:09 AM UTC; Query executed in 0.01 m

,ProfileID,Name,MemberSince,LastUpdate,School,Status,Sex,Birthday,AIM,Website,PoliticalViews,Geography,HighSchool,HomeTown,HomeState,Residence,CurrentAddress,CurrentTown,CurrentState,ProfileID,Status
0,800002,Brian Whitton,2004-03-22,2006-01-11,NYU '07,Undergrad,Male,1984-12-16,bedubbs1,http://www.flickr.com/photos/dubbs,Libertarian,None,George W.Hewlett High School,"Valley Stream, NY",None,None,None,None,None,800002,Single
1,800004,Sunny Kim,2004-03-22,2006-01-06,NYU '07,Undergrad,Female,1985-08-08,sunnyi3unny,http://www.livejournal.com/~sunnyi3unny,Conservative,None,West Windsor Plainsboro High School South '03,"Plainsboro, NJ",None,Thirteenth Street 5AN,None,None,None,800004,Single
2,800005,Ariel Podwal,2004-03-22,2006-01-16,NYU '07,Undergrad,Male,1985-05-29,ClayBandit,http://www.twoshotsofrye.com,None,None,Harrison High School '03,"Harrison, NY",None,Palladium 524D,None,None,None,800005,Single
3,800008,Stephanie Lai,2004-03-22,2005-12-22,NYU '07,Undergrad,Female,1985-06-12,socks4414,None,Liberal,None,Millburn Senior High School '03,"Short Hills, NJ",None,Carlyle Court,None,None,None,800008,Single
4,800010,Jeff Chlystun,2004-03-22,2006-01-16,NYU '06,Faculty,Male,1983-12-20,RedGiraffeBot,http://www.watchstevelewsuckthediack.com,Apathetic,None,Red Hook Senior High School '02,"Red Hook, NY 12571",None,None,None,None,None,800010,Single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7867,834604,Sao Her,2006-01-17,2006-01-17,NYU,Undergrad,Female,1985-08-04,None,None,Other,None,Spring Lake Park Senior High School '03,"Blaine (Minneapolis), MN 55449",None,Lafayette Street,None,None,None,834604,Single
7868,834605,Haby Barry,2006-01-17,2006-01-17,NYU '06,Undergrad,Female,1986-09-21,naughtygirl9216,None,Liberal,None,None,"Tobyhanna, PA 18466",None,Broome Street 706A,"400 Broome Street Apt. 706A-2 New York, NY 10013",None,None,834605,Single
7869,834614,Noah Shibley,2006-01-17,2006-01-17,NYU '06,Grad Student,Male,NaT,None,None,Very Liberal,"New York, NY",None,None,None,None,None,None,None,834614,Single
7870,834628,Ryan 'Millionaire' Jeffries,2005-09-18,2006-01-17,NYU '04,Alumnus/Alumna,Male,NaT,None,None,None,"New York, NY",De La Salle Institute '00,"Chicago, IL 60653",None,None,None,NYC,None,834628,Single


#### List all the Single students who live in Palladium (as declared in the “Residence” attribute).  Allow for flexible matching of the “Residence” as people list Palladium in different manner (e.g “Palladium 101” vs “Palladium”)






In [87]:
%%read_sql
SELECT *
FROM Profiles P
  INNER JOIN Relationship R ON R.ProfileID = P.ProfileID
WHERE R.Status = 'Single' AND P.Residence LIKE 'Palladium%%'

Query started at 03:09:10 AM UTC; Query executed in 0.01 m

,ProfileID,Name,MemberSince,LastUpdate,School,Status,Sex,Birthday,AIM,Website,PoliticalViews,Geography,HighSchool,HomeTown,HomeState,Residence,CurrentAddress,CurrentTown,CurrentState,ProfileID,Status
0,800005,Ariel Podwal,2004-03-22,2006-01-16,NYU '07,Undergrad,Male,1985-05-29,ClayBandit,http://www.twoshotsofrye.com,None,None,Harrison High School '03,"Harrison, NY",None,Palladium 524D,None,None,None,800005,Single
1,800012,Josh Fern,2004-03-22,2006-01-16,NYU '07,Undergrad,Male,1985-10-03,GameSetMatch108,None,Liberal,None,Fox Lane High School '03,"Bedford, NY 10506",None,Palladium 524F,None,None,None,800012,Single
2,800033,Santosh Sateesh,2004-03-22,2006-01-17,NYU '07,Undergrad,Male,1985-04-25,JTMarlin909,None,Liberal,None,pine view,"Sarasota, FLORIDA",None,Palladium 603,None,None,None,800033,Single
3,800084,McCoy Jen,2004-03-23,2006-01-17,NYU '07,Undergrad,Male,1985-06-15,ultrarogue,None,Moderate,None,Homestead '03,None,WI,Palladium 824,None,None,None,800084,Single
4,800142,Kristi Satterwhite,2004-03-24,2005-12-18,NYU '07,Undergrad,Female,1985-04-18,starlightbegins,None,None,None,Lawton Chiles High School '03,"Tallahassee, FL 32312",None,Palladium,None,None,None,800142,Single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,830963,Julie Tower,2005-10-04,2005-11-09,NYU '06,Undergrad,Female,1984-08-15,julesnyu,None,Liberal,None,Allen High School '02,"Allen, TX",None,Palladium,None,None,None,830963,Single
280,831635,Chrystina Castellon,2005-10-22,2005-11-06,NYU '06,Undergrad,Female,1984-08-19,lakerdiva02,None,None,None,Narbonne High School,None,CA,Palladium 807,None,None,None,831635,Single
281,831687,Chike Chukwuma,2005-10-23,2005-11-16,NYU,Alumnus/Alumna,Male,1981-06-06,Bronx99,None,Liberal,"New York, NY",Central Bucks High School - West '99,"Doylestown, PA 18901",None,Palladium,None,None,None,831687,Single
282,831807,Michitaka Koike,2005-10-26,2005-10-26,NYU '07,Undergrad,Male,1974-03-02,None,None,Moderate,None,Mercer Island Senior High School '91,"Tokyo, Japan",None,Palladium 1328,None,None,None,831807,Single


#### List all the Single students LookingFor “random play”. List their AIM and their gender in the results.



In [88]:
%%read_sql
SELECT P.AIM, P.Sex
FROM Profiles P
  INNER JOIN Relationship R ON R.ProfileID = P.ProfileID
  INNER JOIN LookingFor L ON L.ProfileID = P.ProfileID
WHERE R.Status = 'Single' AND L.LookingFor = 'Random Play'

Query started at 03:09:11 AM UTC; Query executed in 0.01 m

,AIM,Sex
0,RedGiraffeBot,Male
1,Quickstep1001,Female
2,iheartjavelinas,Male
3,JTMarlin909,Male
4,SVENNN,Male
...,...,...
1368,mariposa943,Female
1369,pfk111,Male
1370,latinafv99,Female
1371,neo7386,Male


#### List all the students who have “The Killers” as favorite Music




In [31]:
%%read_sql
SELECT P.*
FROM Profiles P
  INNER JOIN FavoriteMusic M ON M.ProfileID = P.ProfileID
WHERE M.Music = 'The Killers'

Query started at 03:52:21 AM UTC; Query executed in 0.01 m

,AIM,Sex
0,sunnyi3unny,Female
1,bazketcase45,Male
2,BluEyes402,Female
3,ajcr333,Male
4,kcramos,Female
...,...,...
1104,None,Female
1105,laurie795,Female
1106,None,Female
1107,mrbean664,Female


#### List all the Finance students who like the book “1984”

In [32]:
%%read_sql
SELECT P.*
FROM Profiles P
  INNER JOIN FavoriteBooks B ON B.ProfileID = P.ProfileID
WHERE B.Book = '1984'

Query started at 03:53:04 AM UTC; Query executed in 0.01 m

,ProfileID,Name,MemberSince,LastUpdate,School,Status,Sex,Birthday,AIM,Website,PoliticalViews,Geography,HighSchool,HomeTown,HomeState,Residence,CurrentAddress,CurrentTown,CurrentState
0,800010,Jeff Chlystun,2004-03-22,2006-01-16,NYU '06,Faculty,Male,1983-12-20,RedGiraffeBot,http://www.watchstevelewsuckthediack.com,Apathetic,None,Red Hook Senior High School '02,"Red Hook, NY 12571",None,None,None,None,None
1,800017,Kelly Quann,2004-03-22,2005-12-21,NYU '07,Undergrad,Female,1984-11-09,None,None,Liberal,None,Bensalem Township High School '03,"Bucks County, PA",None,Carlyle Court C13G,None,None,None
2,800027,Dave Birinyi,2004-03-22,2005-12-21,NYU '07,Undergrad,Male,1985-07-12,iheartjavelinas,http://homepages.nyu.edu/~dlb287/,Liberal,None,Garfield High School '03,"Seattle, WA 98119",None,None,"393 Hewes St, Apt 1E Brooklyn, NY 11211",None,None
3,800078,Maribelle Kwong,2004-03-23,2006-01-16,NYU '07,Undergrad,Female,1984-10-25,belle2033,None,None,None,Cathedral High School '03,"Elmhurst, NY 11373",None,None,None,None,None
4,800120,Nick Herman,2004-03-24,2005-09-10,NYU '06,Undergrad,Male,1984-06-24,ASHnick022,None,Liberal,None,Hastings Senior High School '02,"hastings, MN 55033",None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,832874,Michelle Ferguson,2005-11-25,2006-01-03,NYU '06,Undergrad,Female,1984-04-11,mangobiscuit,None,Moderate,None,Wasatch Academy '02,Hong Kong,None,Second Street,"1 East Second Street New York, NY 10003",None,None
721,832906,Ila Hess Luciano,2005-11-26,2005-12-06,NYU '06,Undergrad,Female,1985-11-22,None,None,Very Liberal,None,United Nations International School '03,"New York, NY",None,None,None,None,None
722,833175,Courtney Balestier,2005-12-03,2005-12-03,NYU '06,Grad Student,Female,1983-05-28,clarkscourtney,None,Liberal,"New York, NY",None,"Morgantown, WV 26505",None,None,None,None,None
723,833227,Joby Thoyalil,2005-12-04,2005-12-18,NYU '04,Grad Student,Male,1982-01-07,kenoby107,None,Liberal,"New York, NY",Connetquot High School,"Ronkonkoma, NY 11779",None,None,None,"Brooklyn, NY 11225",None


## Self Joins

#### Switch to IMDb

In [89]:
%%read_sql
USE imdb;  

Query started at 03:09:11 AM UTC; Query executed in 0.01 m

#### List the movies in the database that have both drama and comedy listed among their genres

In [90]:
%%read_sql
SELECT *
FROM movies M
  INNER JOIN movies_genres G1 ON G1.movie_id = M.id
  INNER JOIN movies_genres G2 ON G2.movie_id = M.id
WHERE G1.genre = 'Drama' AND G2.genre = 'Comedy'

Query started at 03:09:11 AM UTC; Query executed in 0.04 m

,id,name,year,rank,movie_id,genre,movie_id,genre
0,21,$5.15/Hr.,2004,NaN,21,Drama,21,Comedy
1,344,...All the Marbles,1981,5.5,344,Drama,344,Comedy
2,359,...Aur Pyaar Ho Gaya,1997,5.5,359,Drama,359,Comedy
3,361,...Because She Disobeyed,1935,6.0,361,Drama,361,Comedy
4,362,"...Belpaese, Il",1977,5.5,362,Drama,362,Comedy
...,...,...,...,...,...,...,...,...
6724,378404,"ltimo amor de Goya, El",1946,NaN,378404,Drama,378404,Comedy
6725,378497,"ltimos das de Pompeyo, Los",1940,NaN,378497,Drama,378497,Comedy
6726,378550,"berfall, Der",2000,7.0,378550,Drama,378550,Comedy
6727,378579,skdar Iskelesi,1960,NaN,378579,Drama,378579,Comedy


#### Switch to Facebook

In [93]:
%%read_sql
USE facebook;  

Query started at 03:09:52 AM UTC; Query executed in 0.01 m

#### List the Profile IDs and for students majoring in computer science and another concentration (Concentration table); show the second concentration as well


In [96]:
%%read_sql
SELECT *
FROM Concentration C1
  INNER JOIN Concentration C2 ON C1.ProfileID = C2.ProfileID
WHERE C1.Concentration = 'Computer Science' AND C2.Concentration != 'Computer Science'

Query started at 03:10:26 AM UTC; Query executed in 0.01 m

,ProfileID,Concentration,ProfileID,Concentration
0,800001,Computer Science,800001,Psychology
1,800012,Computer Science,800012,Hellenic Studies
2,800183,Computer Science,800183,History
3,800184,Computer Science,800184,Cinema Studies
4,800245,Computer Science,800245,Communication Studies
...,...,...,...,...
230,833365,Computer Science,833365,Economics
231,833719,Computer Science,833719,Finance
232,834314,Computer Science,834314,Management and Organizational Behavior
233,834491,Computer Science,834491,Creative Writing


## Outer Joins

#### Switch to IMDB

In [21]:
%%read_sql 
USE imdb;

Query started at 03:36:55 AM UTC; Query executed in 0.01 m

#### List all the movies without actors

In [26]:
%%read_sql
SELECT M.*, R.*
FROM movies M
  LEFT JOIN roles R ON M.id = R.movie_id
WHERE R.movie_id IS NULL

Query started at 03:37:40 AM UTC; Query executed in 0.20 m

,id,name,year,rank,actor_id,movie_id,role
0,1,"#7 Train: An Immigrant Journey, The",2000,NaN,None,None,None
1,8,"$10,000 Under a Pillow",1921,NaN,None,None,None
2,10,"$100,000 Pyramid, The",2001,NaN,None,None,None
3,13,$21 a Day Once a Month,1941,NaN,None,None,None
4,29,"$500,000 Reward",1911,NaN,None,None,None
...,...,...,...,...,...,...,...
88012,412297,"""re for krone""",1998,NaN,None,None,None
88013,412309,"""ltimo Inverno, O""",1953,NaN,None,None,None
88014,412310,"""ltimo Inverno, O""",1957,NaN,None,None,None
88015,412313,"""nica Verdade, A""",1958,NaN,None,None,None


#### List all the movies without an associated genre

In [18]:
%%read_sql
SELECT M.*
FROM movies M
  LEFT JOIN movies_genres G ON M.id = G.movie_id
WHERE G.movie_id IS NULL

Query started at 03:35:09 AM UTC; Query executed in 0.02 m

,id,name,year,rank
0,0,#28,2002,NaN
1,3,"$1,000 Reward",1913,NaN
2,4,"$1,000 Reward",1915,NaN
3,7,"$1,000,000 Reward, The",1920,NaN
4,16,"$30,000",1920,NaN
...,...,...,...,...
137122,412316,"""zem blch krlu""",1991,NaN
137123,412317,"""rgammk""",1995,NaN
137124,412318,"""zgnm Leyla""",2002,NaN
137125,412319,""" Istanbul""",1983,NaN


#### List all the Students that have not listed a Concentration

In [27]:
%%read_sql 
USE facebook;

Query started at 03:37:59 AM UTC; Query executed in 0.01 m

In [29]:
%%read_sql
SELECT P.*, C.*
FROM Profiles P
  LEFT JOIN Concentration C ON P.ProfileID = C.ProfileID
  WHERE C.ProfileID IS NULL 

Query started at 03:38:51 AM UTC; Query executed in 0.01 m

,ProfileID,Name,MemberSince,LastUpdate,School,Status,Sex,Birthday,AIM,Website,PoliticalViews,Geography,HighSchool,HomeTown,HomeState,Residence,CurrentAddress,CurrentTown,CurrentState,ProfileID,Concentration
0,800010,Jeff Chlystun,2004-03-22,2006-01-16,NYU '06,Faculty,Male,1983-12-20,RedGiraffeBot,http://www.watchstevelewsuckthediack.com,Apathetic,None,Red Hook Senior High School '02,"Red Hook, NY 12571",None,None,None,None,None,None,None
1,800014,Yoome Kim,2004-03-22,NaT,NYU,Undergrad,None,NaT,None,None,None,None,None,None,None,None,None,None,None,None,None
2,800016,Anna Gekker,2004-03-22,2006-01-17,NYU '07,Undergrad,Female,1985-11-03,Frick321,None,Moderate,None,Brooklyn Technical High School '03,"Brooklyn, NY",None,Greenwich Hotel,None,None,None,None,None
3,800042,Anthony Spinelli,2004-03-22,2006-01-17,NYU '06,Undergrad,Male,NaT,scodalupo,None,None,None,None,"Morganville, NJ",None,Water Street 1712,None,None,None,None,None
4,800059,Lisa Herman,2004-03-23,NaT,NYU,Undergrad,None,NaT,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6527,834637,Jen Slovin,2006-01-18,2006-01-18,NYU '08,Undergrad,Female,1985-12-17,None,None,None,None,Worcester Academy '04,"Westborough, MA 01581",None,Water Street,None,None,None,None,None
6528,834638,Kumi Dikengil,2006-01-18,NaT,NYU,Alumnus/Alumna,None,NaT,None,None,None,None,None,None,None,None,None,None,None,None,None
6529,834639,Sky Gaven,2006-01-19,NaT,NYU,Undergrad,None,NaT,None,None,None,None,None,None,None,None,None,None,None,None,None
6530,834640,Eli Brown,2005-12-27,NaT,NYU,Grad Student,None,NaT,None,None,None,None,None,None,None,None,None,None,None,None,None
